# NHIRS usage statistics

This notebook provides a first-pass analysis of NHIRS geoserver request logs, to enable NHI to analyse who is using the service, how they are connecting, and trends in use. 

The log file is comprised of a timestamp and a JSON message that contains information on the request, which needs to be parsed to separate out the useful information.

### Notes:

* Presently there's only one dump of the log files from early September 2024. There's a [JIRA ticket with the Flying Hellfish](https://gajira.atlassian.net/browse/NHIRS-218) to create a process to save the logs on a regular basis.


### Requirements

* pandas
* geopandas
* cartopy
* seaborn


### Last updated:

Craig Arthur
September 2024

In [ ]:
import pandas as pd
import geopandas as gpd
import json
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from urllib.request import urlopen
import urllib.parse

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

PROJECTION = ccrs.PlateCarree(central_longitude=180)
TRANSFORM = ccrs.PlateCarree()

#states = cfeature.NaturalEarthFeature(
#    category='cultural',
#    name='admin_1_states_provinces_lines',
#    scale='10m',
#    facecolor='none')
#LAND = cfeature.LAND()

In [ ]:
filename = r"C:\Users\u12161\Downloads\NHIRS geoserver request logs.csv"


We restrict to a subset of the URIs, so we focus on those that are requesting the geoserver endpoints. 

In [ ]:

uris = ['/geoserver/nhirs/ows', '/geoserver/nhirs/wfs', '/geoserver/nhirs/wms',
        '/geoserver/access/ows', '/geoserver/access/wfs', '/geoserver/access/wms']

Define a collection of functions to manipulate the messages. Some of these extract information from the JSON message, others clean strings to remove HTML ascii codes that appear in some parts of the messages. 

In [ ]:
def clean_string(string):
    cleaned = string.replace('%27', '')
    cleaned = re.sub(r"^'|'$", '', cleaned)
    return cleaned


In [ ]:
def parse_message(message):
    # Initialize an empty dictionary
    parsed_dict = {}
    
    # Split the message by '&' to get key-value pairs
    pairs = message.split('&')
    
    # Loop through each pair and split by '=' to get key and value
    for pair in pairs:
        if '=' in pair:
            key, value = pair.split('=', 1)  # Split by the first '='
            # Decode the value in case it's URL encoded
            decoded_value = urllib.parse.unquote(value)
            parsed_dict[key] = decoded_value
        else:
            parsed_dict[pair] = None  # Handle cases where there's no '=' (if needed)
    
    return parsed_dict

def extractLayer(msg):
    try:
        return parse_message(msg)["LAYERS"]
    except KeyError:
        return None

def extractService(msg):
    try:
        return parse_message(msg)["SERVICE"]
    except KeyError:
        return None

def extractEvent(msg):
    try:
        cqlquery = parse_message(msg)["CQL_FILTER"]
    except:
        return None
    else:
        try:
            eventid = parse_message(cqlquery)['event_id']
            return clean_string(eventid)
        except:
            return None

In [ ]:
df = pd.read_csv(filename)
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['event_id'] = df['query_string'].apply(extractEvent)
df['layer'] = df['query_string'].apply(extractLayer)
df['hour'] = df['datetime'].dt.floor("h")
df['day'] = df['datetime'].dt.floor('d')
# Select the web service URIs:
df = df.loc[df['uri'].isin(uris)]

# Filter out my own IP (in fact this should be all GA addresses):
df = df.loc[~df['request_ip'].str.startswith('124.47')]

In [ ]:
df.head()

Number of hits from each unique IP address

In [ ]:
df['request_ip'].value_counts()

Number of requests from different applications. 

In [ ]:
df['user_agent'].value_counts()

Plot a chart of the hourly rate of requests across all endpoints

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.lineplot(df.set_index('datetime').resample('h').size(), ax=ax, ) #marker='o', markersize=5)
ax.set_ylabel("# requests per hour")
ax.set_xlabel("Time (UTC)")
dateloc = mdates.DayLocator(interval=5)
datefmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_locator(dateloc)
ax.grid()
fig.tight_layout()

Plot a stacked bar chart of the daily requests, with separate categories for each URI

In [ ]:
dfgroup = df.groupby(['day', 'uri']).size().reset_index(name='request_count')
df_pivot = dfgroup.pivot(index='day', columns='uri', values='request_count').fillna(0)
df_pivot.index = df_pivot.index.date

fig, ax = plt.subplots(1, 1,figsize=(12, 6))
ax = df_pivot.plot(kind='bar', stacked=True, ax=ax, rot=90)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Requests')
ax.set_title('Number of Requests per day by URI')
dateloc = mdates.DayLocator(interval=2)
datefmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_locator(dateloc)
plt.legend(title='URI', ncols=2)
ax.grid()
plt.tight_layout()
plt.show()

In [ ]:
df.groupby(['user_agent', 'request_ip']).size().reset_index(name='application count').to_csv(r"X:\georisk\HaRIA_B_Wind\projects\NHIRS\2. DATA\1. Work Unit Assessment\Usage\20241016_Agent_IP.csv", index=False)

Plot a chart of the daily number of unique IP adresses requesting services

In [ ]:
# Plot number of unique IP addresses by day:
dfgroup = df.groupby(['day'])['request_ip'].nunique().reset_index(name='IP count').set_index('day')
dfgroup.index = dfgroup.index.date

fig, ax = plt.subplots(1, 1,figsize=(12, 6))
ax = dfgroup.plot(kind='bar', stacked=False, ax=ax, rot=0)
ax.set_xlabel('Date')
ax.set_ylabel('IP count')
ax.set_title('Number of unique IPs')
dateloc = mdates.DayLocator(interval=5)
datefmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_locator(dateloc)
ax.grid()
plt.tight_layout()
plt.show()

In [ ]:
def extractLocation(ip):
    try:
        res = urlopen('https://ipinfo.io/' + ip + '/json')
        data = json.load(res)
        city = data['city']
        lon = float(data['loc'].split(',')[0])
        lat = float(data['loc'].split(',')[1])
        return (city, lon, lat)
    except json.JSONDecodeError:
        return None

Next we discover the location of the list of IP addresses so we can plot a map of where the services are being used. 

_Take care with this, as it may raise an access denied error if you provide too long a list_. This makes a request to an IP information page, and if we poll too many times in too short a period, the requests will be blocked. 

In [ ]:
tmpdf = df['request_ip'].value_counts().reset_index()['request_ip'].apply(extractLocation, )
locdf = pd.DataFrame(tmpdf.tolist(), index=tmpdf.index, columns=['city', 'latitude', 'longitude'])
locdf['request_ip'] = df['request_ip'].value_counts().index
locgdf = gpd.GeoDataFrame(data=locdf, geometry=gpd.points_from_xy(locdf['longitude'], locdf['latitude']), crs=TRANSFORM)

In [ ]:
df['request_ip'].value_counts()

Plot a map of the IP locations:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8),
                       subplot_kw={'projection': PROJECTION})

locgdf.plot(marker="*", color='red', markersize=40, ax=ax, transform=TRANSFORM)
ax.coastlines()
#ax.add_feature(states, edgecolor='0.15', linestyle='--')
#ax.add_feature(cfeature.LAND, edgecolor='k')
gl = ax.gridlines(draw_labels=True)
ax.set_extent((110, 160, -45, -10), crs=TRANSFORM) # Change this if you want to plot global map
fig.tight_layout()


Pivot table of the applications ('agent') that are used by each unique IP. Typically, there's only one application, but some IP addresses report more than one application, suggesting multiple users are making requests.

In [ ]:
df.pivot_table(index='request_ip', columns='user_agent', values='day', aggfunc='count').fillna(0)

In [ ]:
df.event_id.unique()

In [ ]:
tmpdf = df.loc[df['event_id'].notnull()]
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.lineplot(tmpdf.set_index('datetime').resample('h').size(), ax=ax, ) #marker='o', markersize=5)
ax.set_ylabel("# requests per hour")
ax.set_xlabel("Time (UTC)")
dateloc = mdates.DayLocator(interval=5)
datefmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_locator(dateloc)
ax.grid()
fig.tight_layout()